необходимо прикрепить к ноутбуку датасеты, которые планируется использовать

part 0	https://www.kaggle.com/datasets/rjhhvbjvb/sova-audiobooks-ru-0

part 1	https://kaggle.com/datasets/c9e52c886e845e74a8d6bf9ad54c6e484f09cb19ad761327892ab875de1c8c77

part 2	https://kaggle.com/datasets/77749f53b714f8bafeace83d7f310d83c0feb108690c99c806f969c10fda3bb2

part 3	https://kaggle.com/datasets/18cfda4e68bf7ad35ebf05b25759f2d1a7b1f951bbe28fc5d2d45e1b8fa72e1a

part 4	https://kaggle.com/datasets/359cd3b0c777ddd80f1613f35043663277205136bb116a1bdd7da1c7d7e138db

part 5	https://kaggle.com/datasets/cdb96c8e39f9b175ef5ba3291582da1b2c27a04caa22b0f75337ce98a3a44a1c

part 6	https://kaggle.com/datasets/999ceb12d94c07f52a8a6b0950ad27550ab30a3c96c60f070495bbe890703ebf

part 7	https://kaggle.com/datasets/7202bd62533e4307decba3cd978fe5cdcc4ecbad90596e1c71593d961a7e13a4

part 8	https://kaggle.com/datasets/a78a2594f770aa27bedf00c43dfd076a1b3b6451b293c5d274573ac0ae59295f

part 9	https://kaggle.com/datasets/cd765360abfa918ed7519293fa3f1f1804c92b48c4d88cc73fe7c1fc2d149bd3

part a	https://kaggle.com/datasets/89a6f2d4489c9ceb7ad2d688f6af16e674d2e9537eb11805d0ce7e85d5bc514b

part b	https://kaggle.com/datasets/c4e5e56fe4f4bd44acce05272a35a5387e5d77d0d3f92b14607f9daaa0d14548

part c	https://kaggle.com/datasets/86a5464c94e4ba497f9f8202c5333db1b5b9b7f67e49001640dce2e5df120a9f

part d	https://kaggle.com/datasets/6ca160acfe806c7bd32b8fe6854fb85dc7719803fa743a3a729f25980f075802

part e	https://kaggle.com/datasets/abf63add76d79c500f8515d24a7737ae433e2503d79cf34557c29840421866a1

part f	https://kaggle.com/datasets/a9802f252453e82d168cd20d6017d65d9feaa4287a06081da89337f6ec189a74

In [ ]:
!pip install jiwer==3.0.1

!pip install datasets==2.10.1

In [2]:
import os
import pandas as pd
import shutil
from datasets import load_dataset
from scipy.io import wavfile

In [ ]:
df_filename = []
df_path_to_wav = []
df_text = []
df_len_of_wav = []

# датасет разеделен на 16 частей 0-f
# нужно задать через запятую те части датасета, которые сейчас используются
nums_of_used_datasets = ['0', '1', '2', '3', '4', '5', '6', '7']

folders = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f']

for num in nums_of_used_datasets:
    print(num)
    for folder in folders:
        print(folder, end=' ')
        print()
        directory = f"/kaggle/input/sova-audiobooks-ru-{num}/{num}/{folder}"
        for filename in os.listdir(directory):
             if filename.endswith(".txt"):
                    txt_filename = filename
                    wav_filename = f"{filename.split('.')[0]}.wav"

                    txt_path = os.path.join(directory, txt_filename)
                    wav_path = os.path.join(directory, wav_filename)

                    df_filename.append(f'train/{wav_filename}')
                    df_path_to_wav.append(wav_path)
                    with open(txt_path,'r') as f:
                        df_text.append(f.read())
                    sr, wav = wavfile.read(wav_path)
                    df_len_of_wav.append(round(len(wav)/sr, 1 ))


df_sova_audiobooks = pd.DataFrame({
    'file_name': df_filename,
    'full_path': df_path_to_wav,
    'answer': df_text,
    'duration': df_len_of_wav,
})
df_sova_audiobooks['source'] = 'sova_audiobooks'
df_sova_audiobooks

In [4]:
df_sova_audiobooks['duration'].sum()/3600

136.84550000000002

In [5]:
os.mkdir("/kaggle/working/sova")
os.mkdir("/kaggle/working/sova/train")

In [ ]:
for index, row in df_sova_audiobooks.iterrows():
    if index % 10000 == 0:
        print(index)
    try:
        shutil.copy(row['full_path'], f"/kaggle/working/sova/{row['file_name']}")
    except:
        print('except', index)
        df_sova_audiobooks.drop(index, inplace=True)

In [ ]:
df_sova_audiobooks.drop(columns=['full_path'], inplace=True)

In [9]:
df_sova_audiobooks.to_csv('/kaggle/working/sova/metadata.csv')

In [ ]:
# создаем датасет локально

dataset = load_dataset("audiofolder", data_dir='/kaggle/working/sova')

In [ ]:
# загружаем на HuggingFace
from huggingface_hub import login

login(token='') # добавить write токен нужного аккаунта HuggingFace 

dataset.push_to_hub(f"aanosov/mfdp_sova_part1", private=True)